# 迁移学习

   实际应用中，很少人会从头开始训练一个卷积神经网络，因为这需要大量的数据样本以及需要长时间的训练。更多情况下是先在一个大的数据集(比如ImageNet)上预训练一个卷积网络，然后以以下两种方式使用预训练好的模型：

- **将预训练的网络作为特征提取器**

  在ImageNet数据集上预训练卷积神经网络，保留前面的卷积、池化层等(可以看成是特征提取器)，移除最后几层的全连接层(可以看成是分类器)。固定住特征提取器，在新的数据集上训练新的分类器(新的全连接层)。

- **微调预训练网络(fine-tuning)**

  这种方式不仅训练最后的分类器，而且需要微调前面的特征提取器。可以微调全部的卷积层，也可以只微调后面的卷积层，因为一般前面的卷积层负责提取一些通用的特征(比如边缘、颜色斑点等特征)，而后续的卷积层提取的特征更针对训练集中的数据。

**问题：如何挑选迁移学习的方式？**

|                    | very similar dataset |very different dataset |
|:-------------------|:---------------------|:----------------------|
|**very little data**| ConvNet as fixed feature extractor, <br>retrain classifier. | You are in trouble. |
| **a lot of data**  | Finetune a few layers. | Finetune a large number of layers. |


## 1. 加载数据集

将oxflowers17数据集分为训练集与数据集，分别加载。

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './flower_dataset/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# 2. 定义训练函数

In [2]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


## 3. 迁移学习

### ３.1 将预训练的网络作为特征提取器

pytorch的torchvision.models集成了许多常见的等深度网络，包括：
- AlexNet
- VGG
- ResNet
- SqueezeNet
- DenseNet
- Inception v3
- GoogLeNet

pytorch还提供了预训练的模型，可以直接使用。
直接加载resnet18，并且设置为不更新参数：

In [2]:
from torchvision import models

model_conv = models.resnet18(pretrained=True)# pretrained设置为True则加载预训练的模型
# 不对参数进行更新
for param in model_conv.parameters():
    param.requires_grad = False
print(model_conv)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

更换输出层：

In [4]:
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 17)
model_conv = model_conv.to(device)

设置损失函数为交叉熵损失：

In [5]:
criterion = nn.CrossEntropyLoss()

定义优化器：

In [6]:
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

调整学习率，每7个epoch按0.1倍衰减学习率：

In [7]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

训练并测试模型：

In [8]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 2.1402 Acc: 0.3992
val Loss: 1.2578 Acc: 0.6529

Epoch 1/9
----------
train Loss: 1.1409 Acc: 0.7471
val Loss: 0.8239 Acc: 0.7765

Epoch 2/9
----------
train Loss: 0.8466 Acc: 0.8059
val Loss: 0.6095 Acc: 0.8176

Epoch 3/9
----------
train Loss: 0.7050 Acc: 0.8336
val Loss: 0.5918 Acc: 0.8118

Epoch 4/9
----------
train Loss: 0.6246 Acc: 0.8521
val Loss: 0.5652 Acc: 0.7882

Epoch 5/9
----------
train Loss: 0.5506 Acc: 0.8647
val Loss: 0.4603 Acc: 0.8588

Epoch 6/9
----------
train Loss: 0.5295 Acc: 0.8681
val Loss: 0.4618 Acc: 0.8588

Epoch 7/9
----------
train Loss: 0.5131 Acc: 0.8714
val Loss: 0.4277 Acc: 0.8706

Epoch 8/9
----------
train Loss: 0.4794 Acc: 0.8874
val Loss: 0.4295 Acc: 0.8824

Epoch 9/9
----------
train Loss: 0.4792 Acc: 0.8866
val Loss: 0.4651 Acc: 0.8647

Training complete in 13m 25s
Best val Acc: 0.882353


### ３.２ 微调预训练网络(fine-tuning)

加载预训练的网络：

In [5]:
model_ft = models.vgg16(pretrained=True)
print(model_ft)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

修改vgg16的最后几层全连接层：

In [10]:
import torch.nn as nn
from torch.nn import Sequential

model_ft.classifier = Sequential(nn.Linear(in_features=25088, out_features=4096, bias=True),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.5),
                                 nn.Linear(in_features=4096, out_features=2048, bias=True),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.5),
                                 nn.Linear(in_features=2048, out_features=17, bias=True))
model_ft = model_ft.to(device)

设置损失函数为交叉熵损失：

In [11]:
criterion = nn.CrossEntropyLoss()

设置优化器。因为模型的前面几层只需微调，最后一层需要重新训练，所以要设置不同的学习率，可以按以下方式设置网络中不同层的学习率：

In [12]:
optimizer_ft = optim.SGD([
                          {'params': model_ft.features.parameters(), 'lr':1e-5},
                          {'params': model_ft.classifier.parameters()},
                         ], lr=0.001, momentum=0.9)

调整学习率，每7个epoch按0.1倍衰减学习率

In [13]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

训练并测试模型：

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.4884 Acc: 0.8723
val Loss: 0.4379 Acc: 0.8706

Epoch 1/9
----------
train Loss: 0.4580 Acc: 0.8849
val Loss: 0.4155 Acc: 0.8647

Epoch 2/9
----------
train Loss: 0.4598 Acc: 0.8924
val Loss: 0.4257 Acc: 0.8824

Epoch 3/9
----------
